# Libraries

In [1]:
import os
import pandas as pd
from datetime import datetime

Semana 11: 08-Marzo-2021 al 14-Marzo-2021

finicio = datetime(2021, 3, 8)
ffin = datetime(2021, 3, 14)

# Importación

In [2]:
%%time
#path = '../../../../../../../../genommalabinternacional/Cristian Javier Sanchez Yepez - NoB2B/Copservir/Copservir Faltante/2021/S 9/'
path = '../../../../1Data/2Catalogue/'
df_pdv = pd.read_csv(path + 'Reporte_Inventario_Ventas_PDV.csv', encoding='latin1')
df_suc = pd.read_excel(path + 'Reporte_Inventario_Ventas_Sucursal.xlsx', sheet_name='INVENTARIOS SUCURSALES', header=4)

Wall time: 583 ms


# Transformación inventarios por sucursal

In [8]:
data_suc = df_suc[['Unnamed: 0', 'Unnamed: 1', 'BQUILLA', 'BOGOTA', 'BMANGA', 'CALI']].copy()

data_suc.rename({'BQUILLA':'BAR', 'BOGOTA':'BOG', 'BMANGA':'BUC', 'CALI':'CAL'}, axis=1, inplace=True)
data_suc.rename(dict(zip(data_suc.columns[:2], data_suc.loc[1].tolist()[:2])), axis=1, inplace=True)

data_suc.drop([0, 1], axis=0, inplace=True)
data_suc.reset_index(drop=True, inplace=True)

data_suc_t = data_suc.melt(id_vars=['Codigo', 'Descripcion'], value_vars=['BAR', 'BOG', 'BUC', 'CAL'], var_name='codHoml', value_name='Cantidad Saldo')

In [24]:
data_suc_t['Mostrador'] = data_suc_t['codHoml'].map({'BAR':'BARRANQUILLA', 'BOG':'BOGOTA', 'BUC':'BUCARAMANGA', 'CAL':'CALI'})
data_suc_t['so'] = 0

In [25]:
data_suc_t

,Codigo,Descripcion,codHoml,Cantidad Saldo,Mostrador,so
0,15512,JABON ASEPXIA NEUTRO BARRA X 100 GRS,BAR,0,BARRANQUILLA,0
1,15706,UNESIA CREMA ANTIMICOTICO TUBO X 20 GR,BAR,327,BARRANQUILLA,0
2,27543,MEDICASP CHAMPU CAPILAR ANTICASPA KETOCONAZOL ...,BAR,559,BARRANQUILLA,0
3,63944,CREMA SECANTE ASEPXIA ANTI IMPERFECCIONES CAMO...,BAR,0,BARRANQUILLA,0
4,63946,JABON ASEPXIA FORTE BARRA X 100 GRS,BAR,0,BARRANQUILLA,0
...,...,...,...,...,...,...
875,13264,POLVO ASEPXIA MATIZANTE,CAL,0,CALI,0
876,13262,CREMA CORPORAL CICATRICURE ESPECIALIZADA REAFI...,CAL,0,CALI,0
877,13261,CREMA CORPORAL CICATRICURE ESPECIALIZADA PREVE...,CAL,0,CALI,0
878,13258,CREMA CORPORAL CICATRICURE ESPECIALIZADA PREVE...,CAL,0,CALI,0


# Transformación inventarios y ventas por PDV

In [27]:
df_pdv.columns

Index(['SUC', 'PDV', 'NOMBRE PDV', 'CIUDAD', 'COD', 'DESCRIPCION',
       'PRESENTACION', 'PRV', 'NOMPROVEEDOR', 'Inv Und Total',
       'Vta Und Total'],
      dtype='object')

In [29]:
data_pdv = df_pdv[['PDV', 'NOMBRE PDV', 'COD', 'DESCRIPCION', 'Inv Und Total', 'Vta Und Total']].copy()

# Concatenación de inventario y stock

In [30]:
def homolog_codigos(codigo):
    
    dict_locales = {'400000000':'4E8', '4000000':'4E6', '400000':'4E5',
                    '40000':'4E4', '4000':'4E3', '400':'4E2', '40':'4E1',
                    '3000000000':'3E9', '300000000':'3E8', '30000000':'3E7',
                    '3000000':'3E6', '300000':'3E5', '30000':'3E4', 
                    '3000':'3E3', '30':'3E1', '200000000':'2E8', '20000000':'2E7',
                    '200000':'2E5', '20000':'2E4', '200':'2E2',  '20':'2E1',
                    '1000000000':'1E9', '100000000':'1E8', '10000000':'1E7',
                    '100000':'1E5', '10000':'1E4', '1000':'1E3', '100':'1E2', '10':'1E1'}
    
    try:
        return dict_locales[codigo]
    except:
        return codigo

In [33]:
data_pdv['PDV'] = data_pdv['PDV'].astype(str)
data_pdv['PDV'] = data_pdv['PDV'].map(homolog_codigos)

In [41]:
data_suc_t.rename({'Codigo':'Código producto', 'Descripcion':'Producto'}, axis=1, inplace=True)

In [42]:
data_pdv.rename({'PDV':'codHoml', 'NOMBRE PDV':'Mostrador', 'COD':'Código producto', 'DESCRIPCION':'Producto', 'Inv Und Total':'Cantidad Saldo', 'Vta Und Total':'so'}, axis=1, inplace=True)

data = pd.concat([data_pdv, data_suc_t])

data['Cantidad Saldo'] = data['Cantidad Saldo'].astype(float)

data = data.pivot_table(index=['codHoml', 'Mostrador', 'Código producto', 'Producto'], values=['Cantidad Saldo', 'so'], aggfunc='sum').reset_index()

In [91]:
data['so'].sum()

45416.92

# Desacumulación

In [65]:
path = '../../../../../../../02Paises/Colombia/Copservir/Layout/'
files = ['Copservir_2021_06.xlsx', 'Copservir_2021_07.xlsx', 'Copservir_2021_08.xlsx']

df_acum_ant = pd.DataFrame()
for f in files:
    df_acum_ant = pd.concat([df_acum_ant, pd.read_excel(path + f)])

In [124]:
data_acum_ant = df_acum_ant.pivot_table(index=['codHoml', 'Mostrador', 'Código producto', 'Producto'], values=['Cantidad Saldo', 'so'], aggfunc='sum').reset_index()

In [116]:
data_acum_ant.shape

(85849, 6)

In [125]:
data_acum_ant['codHoml'] = data_acum_ant['codHoml'].astype(str)
data_acum_ant['Código producto'] = data_acum_ant['Código producto'].astype(str)
data_acum_ant['ID'] = data_acum_ant['codHoml'] + data_acum_ant['Código producto']

In [126]:
%%time
ids_reps = data_acum_ant['ID'].value_counts()[data_acum_ant['ID'].value_counts() > 1].index.tolist()
for i in ids_reps:
    nombres_sku = data_acum_ant.loc[data_acum_ant['ID'] == i, 'Producto'].unique().tolist()
    mostradores = data_acum_ant.loc[data_acum_ant['ID'] == i, 'Mostrador'].unique().tolist()
    data_acum_ant.loc[data_acum_ant['ID'] == i, 'Producto'] = nombres_sku[-1]
    data_acum_ant.loc[data_acum_ant['ID'] == i, 'Mostrador'] = mostradores[0]

Wall time: 1min 56s


In [127]:
data_acum_ant = data_acum_ant.pivot_table(index=['codHoml', 'Mostrador', 'Código producto', 'Producto'], values=['Cantidad Saldo', 'so'], aggfunc='sum').reset_index()

In [128]:
data_acum_ant.shape

(82212, 6)

In [129]:
data_acum_ant['ID'] = data_acum_ant['codHoml'] + data_acum_ant['Código producto']

In [131]:
data['codHoml'] = data['codHoml'].astype(str)
data['Código producto'] = data['Código producto'].astype(str)
data['ID'] = data['codHoml'] + data['Código producto']

In [132]:
data_desac = pd.merge(data, data_acum_ant, on='ID', how='left')

In [133]:
data_desac['so_x'].sum()

45416.92

In [134]:
data_desac['so_y'].sum()

34736.510485925995

In [136]:
data_desac['so'] = data_desac['so_x'] - data_desac['so_y']

In [147]:
data_desac['so'].sum()

10617.409514074001

In [146]:
data_desac['so'].apply(lambda x: 0 if x < 0 else x).sum()

12873.971250061

In [137]:
data_desac.columns

Index(['codHoml_x', 'Mostrador_x', 'Código producto_x', 'Producto_x',
       'Cantidad Saldo_x', 'so_x', 'ID', 'codHoml_y', 'Mostrador_y',
       'Código producto_y', 'Producto_y', 'Cantidad Saldo_y', 'so_y', 'so'],
      dtype='object')

In [138]:
data_desac[['codHoml_x', 'Código producto_x', 'so_x', 'so_y',  'so']]

,codHoml_x,Código producto_x,so_x,so_y,so
0,101,15706,3.0,3.25,-0.25
1,101,27543,3.0,3.25,-0.25
2,101,63947,0.0,0.00,0.00
3,101,63956,1.0,0.00,1.00
4,101,64629,0.0,0.00,0.00
...,...,...,...,...,...
81949,CAL,147035,0.0,0.00,0.00
81950,CAL,147036,0.0,0.00,0.00
81951,CAL,147037,0.0,0.00,0.00
81952,CAL,147038,0.0,0.00,0.00


In [142]:
df_acum_ant[(df_acum_ant['Código producto'] == 15706)&(df_acum_ant['codHoml'] == '101')]

,codHoml,Mostrador,Código producto,Producto,Cantidad Saldo,so
71386,101,REBAJA PLUS No. 16 BARRANQUILLA PLAZA DE MERCADO,15706,UNESIA CREMA ANTIMICOTICO TUBO X 20 GR,3.0,1.25
10728,101,REBAJA PLUS No. 16 BARRANQUILLA PLAZA DE MERCADO,15706,UNESIA CREMA ANTIMICOTICO TUBO X 20 GR,1.0,2.00
71287,101,REBAJA PLUS No. 16 BARRANQUILLA PLAZA DE MERCADO,15706,UNESIA CREMA ANTIMICOTICO TUBO X 20 GR,3.0,0.00


# Transformación de Inventarios

Primero transformamos el *Inventario por PDV (ip)*:

In [4]:
data_ip = df_ip[['Mostrador', 'Código producto', 'Producto', 'Cantidad Saldo']].copy()
data_ip['codHoml'] = df_ip['Mostrador'].apply(lambda x: x.split('-')[0])
data_ip['so'] = 0

Luego, transformamos el *inventario consolidado (ic)*:

In [5]:
data_ic = df_ic[['Código producto', 'Producto', 'Cantidad Saldo.1', 'Cantidad Saldo.2', 'Cantidad Saldo.3', 'Cantidad Saldo.4']].copy()
data_ic.rename({'Cantidad Saldo.1':'BAR', 'Cantidad Saldo.2':'BOG', 'Cantidad Saldo.3':'BUC', 'Cantidad Saldo.4':'CAL'}, axis=1, inplace=True)
# Despivoteamos las columnas y lo guardamos en data_ic_t
data_ic_t = data_ic.melt(id_vars=['Código producto', 'Producto'], value_vars=['BAR', 'BOG', 'BUC', 'CAL'])

In [6]:
data_ic_t.rename({'value':'Cantidad Saldo', 'variable':'codHoml'}, axis=1, inplace=True)
data_ic_t['Mostrador'] = data_ic_t['codHoml'].map({'BAR':'BARRANQUILLA', 'BOG':'BOGOTA', 'BUC':'BUCARAMANGA', 'CAL':'CALI'})
data_ic_t['so'] = 0

Al final concatenamos ambos inventarios en unos solo:

In [7]:
data_inv = pd.concat([data_ip, data_ic_t])

Y limpiamos algunas columnas

In [8]:
data_inv.loc[data_inv['Cantidad Saldo'] == '-', 'Cantidad Saldo'] = 0

data_inv['Cantidad Saldo'] = data_inv['Cantidad Saldo'].astype(float)

# Transformación de Ventas

In [9]:
def ftarget(fecha_inic, fecha_fin):
    
    meses = {"01":'Ene', "02":'Feb', "03":'Mar',
             "04":'Abr', "05":'May', "06":'Jun',
             "07":'Jul', "08":'Ago', "09":'Sep',
             "10":'Oct', "11":'Nov', "12":'Dic'}
    
    range_fechas = pd.date_range(finicio, ffin).tolist()
    fechas_list = []
    for f in range_fechas:
        fechas_list.append(str(f.day).zfill(2) + '_' + meses[str(f.month).zfill(2)] + '_' + str(f.year)[-2:])
    return fechas_list

In [10]:
cols_datos = ftarget(finicio, ffin)

In [11]:
data_vd = df_vd[['Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Día'] + cols_datos].copy()

data_vd.replace('-', 0, inplace=True)

data_vd['so'] = data_vd[cols_datos].sum(axis=1)

data_vd.rename(dict(zip(data_vd.columns[:-1], data_vd.loc[0].tolist()[:-1])), axis=1, inplace=True)

In [12]:
data_vd.drop(['Cantidad'], axis=1, inplace=True)

data_vd.drop(0, axis=0, inplace=True)
data_vd.reset_index(inplace=True, drop=True)

data_vd['Cantidad Saldo'] = 0

# Concatenación de inventario y stock

In [13]:
def homolog_codigos(codigo):
    
    dict_locales = {'400000000':'4E8', '4000000':'4E6', '400000':'4E5',
                    '40000':'4E4', '4000':'4E3', '400':'4E2', '40':'4E1',
                    '3000000000':'3E9', '300000000':'3E8', '30000000':'3E7',
                    '3000000':'3E6', '300000':'3E5', '30000':'3E4', 
                    '3000':'3E3', '30':'3E1', '200000000':'2E8', '20000000':'2E7',
                    '200000':'2E5', '20000':'2E4', '200':'2E2',  '20':'2E1',
                    '1000000000':'1E9', '100000000':'1E8', '10000000':'1E7',
                    '100000':'1E5', '10000':'1E4', '1000':'1E3', '100':'1E2', '10':'1E1'}
    
    try:
        return dict_locales[codigo]
    except:
        return codigo

In [14]:
data_inv['codHoml'] = data_inv['codHoml'].astype(str)
data_inv['codHoml'] = data_inv['codHoml'].map(homolog_codigos)
data_inv['Código producto'] = data_inv['Código producto'].astype(str)
data_inv['ID'] = data_inv['codHoml'] + data_inv['Código producto']

In [15]:
data_vd['CodigoComercial'] = data_vd['CodigoComercial'].astype(str)
data_vd['CodigoComercial'] = data_vd['CodigoComercial'].map(homolog_codigos)
data_vd['Código producto'] = data_vd['Código producto'].astype(str)
data_vd['ID'] = data_vd['CodigoComercial'] + data_vd['Código producto']

In [44]:
data = pd.merge(data_inv, data_vd, on='ID', how='outer')

In [26]:
fill = lambda col1, col2: col1 if pd.isnull(col2) else (col2 if pd.isnull(col1) else col1)

In [45]:
%%time
data['Código producto'] = data['Código producto_y'].combine(data['Código producto_x'], fill)
data['Cantidad Saldo'] = data['Cantidad Saldo_x'].combine(data['Cantidad Saldo_y'], fill)
data['so'] = data['so_y'].combine(data['so_x'], fill)
data['Mostrad'] = data['Nombre Mostrador'].combine(data['Mostrador'], fill)
data['codHomol'] = data['CodigoComercial'].combine(data['codHoml'], fill)
data['Product'] = data['Nombre Producto'].combine(data['Producto'], fill)

Wall time: 14.3 s


In [46]:
data.drop(['codHoml', 'Mostrador', 'Código producto_x', 'Producto',
       'Cantidad Saldo_x', 'so_x', 'ID', 'CodigoComercial', 'Nombre Mostrador',
       'Código producto_y', 'Nombre Producto', 'so_y', 'Cantidad Saldo_y'], axis=1, inplace=True)

data.rename({'Mostrad':'Mostrador', 'codHomol':'codHoml', 'Product':'Producto'}, axis=1, inplace=True)

# Exportación de datos

In [51]:
data[['codHoml', 'Mostrador', 'Código producto', 'Producto', 'Cantidad Saldo', 'so']].to_excel('../../../../../../../02Paises/Colombia/Copservir/Copservir_2021_11.xlsx', index=False, sheet_name='2021_11')